# Task 3: Decrypting a Message (Decapsulation & Symmetric Decryption)

## Learning Objectives

Upon completing this task, you should be able to:

1.  **Understand the role of the recipient** in a public-key cryptosystem when receiving an encrypted message.
2.  **Apply** the concept of **Key Decapsulation** using ML-KEM and the recipient's private key to securely retrieve a shared secret.
3.  **Recognize** that successful decapsulation using the correct private key and encapsulated key yields the *same* shared secret that the sender originally generated.
4.  **Understand** how this retrieved shared secret, along with the nonce from the sender, is used with a **symmetric cipher** (AES-GCM) to decrypt the message.
5.  **Identify** that if any of the inputs (private key, encapsulated key, nonce, or encrypted message) are incorrect or mismatched, decryption will fail.
6.  **Interact** with the Python script to perform decapsulation and symmetric decryption.
7.  **Verify** the integrity and confidentiality of the communication by successfully retrieving the original plaintext message.

## Understanding the Task and the Code

As the recipient, you have received three pieces of information from the sender (Task 2):
1.  The **Encapsulated Key** (`ciphertext_kem`).
2.  The **Nonce** (for AES-GCM).
3.  The **Encrypted Message** (ciphertext from AES-GCM).

You also have your **Private Key** from Task 1. Your goal is to use these items to recover the original plaintext message.

1.  **Key Decapsulation (ML-KEM):** You'll use your **private key** and the sender's **Encapsulated Key** (`ciphertext_kem`). The ML-KEM decapsulation process will "open" the `ciphertext_kem`. If successful (i.e., if your private key is the correct counterpart to the public key used by the sender for encapsulation), this process will yield the *exact same* shared secret that the sender generated in Task 2.
2.  **Symmetric Decryption (AES-GCM):** Once you have recovered the shared secret, you'll use it as the key for AES-GCM, along with the **Nonce** provided by the sender, to decrypt the **Encrypted Message**. If the key, nonce, and encrypted message are all correct and untampered, AES-GCM will return the original plaintext message. AES-GCM also provides authentication, meaning it can detect if the ciphertext was modified after encryption.

**Decryption Process (`on_decrypt_button_clicked` function):**

*   **Input Validation:** Checks if all required fields are filled.
*   **Decode Inputs:** All Base64 encoded inputs (private key, encapsulated key, nonce, encrypted message) are decoded back into their raw byte formats.
*   **KEM Decapsulation:**
    *   `with oqs.KeyEncapsulation(selected_kem_alg, private_key_bytes) as kem_for_decapsulation:`: Initializes a KEM object. Crucially, the `private_key_bytes` are provided directly to the constructor. This "primes" the KEM object with your secret key.
    *   `shared_secret_kem = kem_for_decapsulation.decap_secret(ciphertext_kem_bytes)`: This is the core decapsulation operation. It uses the private key (now part of `kem_for_decapsulation`) to process `ciphertext_kem_bytes`. If successful, `shared_secret_kem` will hold the symmetric key. If it fails (e.g., wrong private key, corrupted ciphertext_kem), an error will be raised by `liboqs`.
*   **Symmetric Decryption (AES-256-GCM):**
    *   `aes_key = shared_secret_kem`: The recovered shared secret is used as the AES key.
    *   `aesgcm = AESGCM(aes_key)`: Initializes the AES-GCM cipher.
    *   `decrypted_payload_bytes = aesgcm.decrypt(nonce_bytes, encrypted_message_bytes, None)`: Attempts to decrypt the message. If the key is wrong, the nonce is incorrect, or the `encrypted_message_bytes` have been tampered with, this will raise an exception (commonly `InvalidTag` for AES-GCM).
    *   The successfully decrypted bytes are then decoded from UTF-8 back into a string.
*   **Display Output:** The decrypted plaintext message is shown. Error messages are displayed in the status area if any step fails.

**Your Interaction:**

1.  Run the Python code cell below.
2.  From the **ML-KEM Algorithm** dropdown, select the *exact same algorithm variant* used in Task 1 and Task 2.
3.  Go back to your Task 1 output. Carefully copy the entire **Base64 encoded Private Key**. Paste it into the "Your Private Key (Base64)" text area in Task 3.
4.  Go back to your Task 2 output. Carefully copy the **Base64 encoded Encapsulated Key**, **Nonce**, and **Encrypted Message**. Paste each of these into their corresponding text areas in Task 3.
5.  Click the "Decrypt Message" button.
6.  If all inputs were correct, you should see the original message you typed in Task 2 appear in the "Decrypted Message" area.
7.  Read the status messages to follow the steps. Try deliberately pasting an incorrect private key or modifying one of the inputs from Task 2 to see how the decryption fails.

## Success!

If you see your original message from Task 2 correctly decrypted, you have successfully completed the entire key generation, encapsulation, symmetric encryption, decapsulation, and symmetric decryption cycle using a Post-Quantum Key Encapsulation Mechanism! This demonstrates the fundamental flow of how PQC KEMs are used to secure communications.

---
**Now, run the code cell below to perform Task 3.**

In [3]:
import oqs
import ipywidgets as widgets
from IPython.display import display, HTML as IPHTML
import base64
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend # Though not explicitly used, good practice if not using default.

# --- KEM Algorithms (should match those offered in Task 1 & 2) ---
supported_kems_task3 = ['ML-KEM-512', 'ML-KEM-768', 'ML-KEM-1024']

# --- UI Elements ---
header_task3 = IPHTML("<h2>Task 3: Decrypt a Message (Decapsulation & Symmetric Decryption)</h2>")
description_task3 = IPHTML("""
<p>In this task, you will act as the <b>recipient</b>. You have received an Encapsulated Key, a Nonce, and an Encrypted Message from the sender (Task 2).</p>
<p>You will use your <b>Private Key</b> (generated in Task 1) to decapsulate the shared secret, and then use that secret with the Nonce to decrypt the message.</p>
<ol>
    <li>Select the <em>same ML-KEM algorithm</em> that was used in Task 1 and Task 2.</li>
    <li>Paste your <b>Base64 encoded Private Key</b> (copied from Task 1) into the designated text area.</li>
    <li>Paste the <b>Base64 encoded Encapsulated Key</b> (copied from Task 2 output) into its text area.</li>
    <li>Paste the <b>Base64 encoded Nonce</b> (copied from Task 2 output) into its text area.</li>
    <li>Paste the <b>Base64 encoded Encrypted Message</b> (copied from Task 2 output) into its text area.</li>
    <li>Click 'Decrypt Message'.</li>
</ol>
<p>If all inputs are correct and correspond to each other, the original plaintext message should appear.</p>
""")

kem_alg_dropdown_task3 = widgets.Dropdown(
    options=supported_kems_task3,
    value=supported_kems_task3[1] if len(supported_kems_task3) > 1 else supported_kems_task3[0], # Default to ML-KEM-768
    description='ML-KEM Algorithm:',
    disabled=False,
    style={'description_width': 'initial'}
)

private_key_b64_input_task3 = widgets.Textarea(
    value='',
    placeholder='Paste your Base64 encoded PRIVATE KEY from Task 1 here.',
    description='Your Private Key (Base64):',
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

encapsulated_key_b64_input_task3 = widgets.Textarea(
    value='',
    placeholder='Paste the Base64 Encapsulated Key from Task 2 here.',
    description='Encapsulated Key (from sender):',
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

nonce_b64_input_task3 = widgets.Textarea(
    value='',
    placeholder='Paste the Base64 Nonce from Task 2 here.',
    description='Nonce (from sender):',
    layout={'width': '95%', 'height': '60px'},
    disabled=False,
    style={'description_width': 'initial'}
)

encrypted_message_b64_input_task3 = widgets.Textarea(
    value='',
    placeholder='Paste the Base64 Encrypted Message from Task 2 here.',
    description='Encrypted Message (from sender):',
    layout={'width': '95%', 'height': '100px'},
    disabled=False,
    style={'description_width': 'initial'}
)

decrypt_button_task3 = widgets.Button(
    description='Decrypt Message',
    button_style='success', # Green for success
    tooltip='Decapsulate shared secret and decrypt the message',
    icon='unlock'
)

decrypted_message_output_task3 = widgets.Textarea(
    value='',
    placeholder='Decrypted plaintext message will appear here.',
    description='Decrypted Message:',
    layout={'width': '95%', 'height': '80px'},
    disabled=True, # Output, so disabled for direct editing
    style={'description_width': 'initial'}
)

status_output_task3 = widgets.Output(layout={'width': '95%'})

# --- Button Click Handler ---
def on_decrypt_button_clicked(b):
    selected_kem_alg = kem_alg_dropdown_task3.value
    private_key_b64 = private_key_b64_input_task3.value
    ciphertext_kem_b64 = encapsulated_key_b64_input_task3.value
    nonce_b64 = nonce_b64_input_task3.value
    encrypted_message_b64 = encrypted_message_b64_input_task3.value

    # Clear previous outputs
    decrypted_message_output_task3.value = ""

    with status_output_task3:
        status_output_task3.clear_output()

        if not selected_kem_alg:
            print("Error: Please select an ML-KEM algorithm.")
            return
        if not private_key_b64.strip():
            print("Error: Please paste your Base64 encoded Private Key from Task 1.")
            return
        if not ciphertext_kem_b64.strip():
            print("Error: Please paste the Base64 Encapsulated Key from Task 2.")
            return
        if not nonce_b64.strip():
            print("Error: Please paste the Base64 Nonce from Task 2.")
            return
        if not encrypted_message_b64.strip():
            print("Error: Please paste the Base64 Encrypted Message from Task 2.")
            return

        print(f"Starting decryption process with {selected_kem_alg}...")

        try:
            # 1. Decode all Base64 inputs
            print("Step 1: Decoding Base64 inputs...")
            private_key_bytes = base64.b64decode(private_key_b64)
            ciphertext_kem_bytes = base64.b64decode(ciphertext_kem_b64)
            nonce_bytes = base64.b64decode(nonce_b64)
            encrypted_message_bytes = base64.b64decode(encrypted_message_b64)
            print("  All inputs decoded successfully.")

            # 2. KEM Decapsulation: Use private key to get the shared secret from the encapsulated key
            print(f"\nStep 2: Decapsulating shared secret using {selected_kem_alg} with your private key...")
            with oqs.KeyEncapsulation(selected_kem_alg, private_key_bytes) as kem_for_decapsulation:
                shared_secret_kem = kem_for_decapsulation.decap_secret(ciphertext_kem_bytes)
            
            print(f"  KEM decapsulation successful.")
            print(f"    Recovered shared secret length: {len(shared_secret_kem)} bytes.")

            # Verify shared secret length (should be 32 for Kyber/ML-KEM)
            if len(shared_secret_kem) != 32:
                print(f"Warning: Recovered KEM shared secret length is {len(shared_secret_kem)} bytes, expected 32 for AES-256.")
                aes_key = (shared_secret_kem + b'\0' * 32)[:32] # Potentially insecure adjustment
            else:
                aes_key = shared_secret_kem

            # 3. Symmetric Decryption (AES-256-GCM)
            print(f"\nStep 3: Symmetrically decrypting the message using AES-256-GCM...")
            aesgcm = AESGCM(aes_key)
            try:
                decrypted_payload_bytes = aesgcm.decrypt(nonce_bytes, encrypted_message_bytes, None) # No AAD
                decrypted_message_str = decrypted_payload_bytes.decode('utf-8')
                print("  Message decrypted successfully!")
            except Exception as e: # Catches InvalidTag from AESGCM, among others
                print(f"Error during AES-GCM decryption: {e}")
                print("  This could be due to an incorrect shared secret (KEM decapsulation failed or wrong keys used), wrong nonce, or tampered ciphertext.")
                decrypted_message_output_task3.value = "DECRYPTION FAILED. Check inputs or see status log."
                return


            # 4. Display the decrypted message
            print(f"\nStep 4: Displaying decrypted message.")
            decrypted_message_output_task3.value = decrypted_message_str
            print("\nDecryption complete! The original message is displayed above.")

        except base64.binascii.Error as e:
            print(f"Error decoding one of the Base64 inputs: {e}. Please ensure they are valid Base64 strings from previous tasks.")
        except oqs.OpenSSLError as e: # This can be raised by OQS during decapsulation failure
            print(f"OQS KEM Decapsulation Error: {e}")
            print("  This often means the private key doesn't match the encapsulated key, the encapsulated key is corrupted, or the wrong KEM algorithm was selected.")
            decrypted_message_output_task3.value = "KEM DECAPSULATION FAILED. Check keys/inputs."
        except Exception as e:
            print(f"An unexpected error occurred during decryption: {e}")
            import traceback
            traceback.print_exc()
            decrypted_message_output_task3.value = "UNEXPECTED ERROR. See status log."


decrypt_button_task3.on_click(on_decrypt_button_clicked)

# --- Display UI ---
display(header_task3)
display(description_task3)

main_ui_container_task3 = widgets.VBox([
    kem_alg_dropdown_task3,
    private_key_b64_input_task3,
    encapsulated_key_b64_input_task3,
    nonce_b64_input_task3,
    encrypted_message_b64_input_task3,
    decrypt_button_task3,
    widgets.HTML("<hr><h3>Decryption Result:</h3>"),
    decrypted_message_output_task3,
    status_output_task3
], layout={'width': '100%'})

display(main_ui_container_task3)